# Реализация модели двустороннего защищенного чата в мессенджере по спецификации Signal

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# КГПСЧ
from Crypto.Random import get_random_bytes, random

# HKDF
from Crypto.Protocol.KDF import HKDF

# Вспомогательные радости
from collections import namedtuple
import dataclasses
import binascii
import pickle

----

In [4]:
def as_hex(blob: bytes):
    """
    Перевод блоба в шестнадцатиричное представление
    """
    
    return binascii.hexlify(blob).decode()

----
## Реализация оберток над криптографическими алгоритмами и примитивами

In [5]:
#
# Описание сути следует смотреть здесь:
# https://signal.org/docs/specifications/x3dh/#publishing-keys
#

KeyBundle = namedtuple('KeyBundle', ['IK',    # Ключ-идентификатор (Identity key)
                                     'SPK',   # Подписанный предключ (Signed prekey)
                                     'Sig',   # Подпись предключа (Prekey signature)
                                     'OPKs',  # Коллекция одноразовых предключей (One-time prekeys)
                                    ])

In [6]:
KeyPair = namedtuple('KeyPair', ['private', 'public'])

In [7]:
Message = namedtuple('Message', ['public_key',  # Открытый ключ DH отправляющего
                                 'id',          # Идентитфикатор сообщения
                                 'data',        # Данные сообщения
                                ])

----

In [8]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes, associated_data: bytes = b''):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = gost3412.GOST3412Kuznechik(self._key)
        cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, associated_data)
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes, associated_data: bytes = b''):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        try:
            encrypter = gost3412.GOST3412Kuznechik(self._key)
            cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())

            ct_with_tag = ciphertext + tag

            return cipher.open(nonce, ct_with_tag, associated_data)
        except ValueError as e:
            return None
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

---

In [9]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [10]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [11]:
class Curve(object):
    """
    Обертка над эллиптической кривой. Для простоты это одна из кривых,
    определенных в ГОСТ 34.10
    """
    
    CURVE = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        
        
    @staticmethod
    def generate_private_key():
        """
        Генерирование закрытого ключа
        """
        
        return gost3410.prv_unmarshal(get_random_bytes(64))
    
    
    @staticmethod
    def get_public_key(private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return gost3410.public_key(Curve.CURVE, private_key)
        
        
    @staticmethod
    def multiply_by_scalar(scalar, point):
        """
        Умножение точки эллиптической кривой на скаляр 
        """

        return Curve.CURVE.exp(scalar, *point)

    
    @staticmethod
    def add_points(lhs, rhs):
        """
        Сложение двух точек эллиптической кривой
        """

        return Curve.CURVE._add(*lhs, *rhs)

----
## Элементы протоколов из спецификации Signal

In [12]:
class KdfChainInternal(object):
    """
    Вспомогательный класс, описывающий KDF-цепочку
    """
    
    def __init__(self, key, key_length):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        self._key_length = key_length
        self._kdf_key = key
    
    
    def reset(self, key):
        """
        Сброс состояния, запись нового ключа в качестве
        первоначального
        """
        
        self._kdf_key = key
        
        
    def _kdf_update(self, data):
        self._kdf_key, message_key = HKDF(self._kdf_key, self._key_length, 
                                          data, Streebog, 2)
        return message_key

In [13]:
class KdfChain(KdfChainInternal):
    """
    Класс, описывающий KDF-цепочку, которая генерирует
    новые ключи только на основе своего состояния
    https://signal.org/docs/specifications/doubleratchet/#kdf-chains
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(KdfChain, self).__init__(key, key_length)
        
        
    def step(self):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(b'\xFF' * 32)

In [14]:
class SymmetricRatchet(KdfChainInternal):
    """
    Симметричный храповик
    Новые ключи вырабатываются на основе выработанного 
    секрета Диффи-Хеллмана
    https://signal.org/docs/specifications/doubleratchet/#symmetric-key-ratchet
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(SymmetricRatchet, self).__init__(key, key_length)
        
        
    def step(self, serialized_dh_key):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(serialized_dh_key)

-----

In [15]:
class DiffieHellmanRatchet(object):
    """
    DH-храповик
    https://signal.org/docs/specifications/doubleratchet/#diffie-hellman-ratchet
    """
    
    def __init__(self):
        """
        Инициализация, выработка начальной ключевой пары
        """
        
        self._current_foreign_public_key = None
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
    
    def is_uninitialized(self):
        """
        Проверка того, что открытый ключ другой стороны отсутствует
        """
        
        return self._current_foreign_public_key is None
    
    
    def initialize(self, public_key):
        """
        Инициализация открытого ключа другой стороны
        """
        
        self._current_foreign_public_key = public_key
        
        
    def is_same_public_key(self, public_key):
        """
        Проверка равенства открытых ключей
        """
        
        if self._current_foreign_public_key is None:
            return False
        
        return public_key == self._current_foreign_public_key
    
    
    def step(self, public_key=None):
        """
        Выработка общего секрета Диффи-Хеллмана
        """
        
        if self._current_key_pair is None:
            raise RuntimeError('Uninitialized DH Ratchet used to produce a shared secret')
        
        if public_key is not None:
            self._current_foreign_public_key = public_key
            
        return Curve.multiply_by_scalar(self._current_key_pair.private, self._current_foreign_public_key)
    
    
    def generate_new_key_pair(self):
        """
        Генерирование новой ключевой пары
        """
        
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._current_key_pair.public
    
    
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return KeyPair(private_key, public_key)

----
## Участники протоколов из спецификации Signal

In [16]:
class SignalServer(object):
    """
    Класс сервера Signal
    """
    
    def __init__(self):
        """
        Инициализация
        """
        
        #
        # Коллекция ключевых данных клиентов, индексируемая по номеру телефона
        #
        
        self._key_bundles = {}
        
        #
        # Коллекция очередей сообщения, индексируемая по паре номеров, 
        # представляющих направление передачи
        #
        
        self._messages = {}
        
        
    def register(self, phone_number: str, keys: KeyBundle):
        """
        Регистрация пользователя по номеру телефона. Сохраняет набор ключей
        """
        
        if phone_number in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is already registered')
            
        trace('[SignalServer]', f'Registered client with phone number {phone_number}')
            
        self._key_bundles[phone_number] = keys
        
        
    def get_public_keys(self, phone_number: str):
        """
        Получение ключей пользователя по его номеру телефона
        """
        
        if phone_number not in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is not registered yet')
            
        #
        # Достану ключи
        #
        
        keys = self._key_bundles[phone_number]
        one_time_keys = keys.OPKs
        
        #
        # Возьму случайный из одноразовых ключей
        #
        
        idx = random.randint(0, len(one_time_keys) - 1)
        result = KeyBundle(keys.IK, keys.SPK, keys.Sig, (one_time_keys.pop(idx), idx))
        
        trace('[SignalServer]', f'Using OPK at {idx}')
        
        #
        # Сохраню обновленный пакет ключей (без одного одноразового)
        #
        
        self._key_bundles[phone_number] = KeyBundle(keys.IK, keys.SPK, keys.Sig, one_time_keys)
        
        return result
    
    
    def push_message(self, phone_from: str, phone_to: str, message: Message):
        """
        Помещение сообщения в очередь
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            self._messages[search_key] = []
            
        trace('[SignalServer]', f'Saved message with id {message.id} from {phone_from} to {phone_to}')
            
        self._messages[search_key].append(message)
        
        
    def peek_message(self, phone_from, phone_to):
        """
        Получение сообщения из очереди
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            return None
        
        if self._messages[search_key] == []:
            return None
        
        return self._messages[search_key].pop(0)

In [17]:
class SignalClient(object):
    """
    Класс клиента Signal
    """
    
    X3DH_MSG_ID      = 'X3DH_MSG'
    DH_RATCHET_PK_ID = 'DH_RATCHET_PK'
    
    
    @dataclasses.dataclass
    class SessionState:
        received: bool
        dh_ratchet: DiffieHellmanRatchet
        root_kdf_chain: KdfChain
        send_ratchet: SymmetricRatchet
        recv_ratchet: SymmetricRatchet
        msg_id: int
    
    
    def __init__(self, phone_number, one_time_keys=10, *, fake_ik=False):
        """
        Инициализация, генерирование ключей
        """
        
        self._phone_number  = phone_number
        
        self._identity_key  = SignalClient._generate_key_pair()
        self._signed_prekey = SignalClient._generate_key_pair()
        self._ot_prekeys    = [SignalClient._generate_key_pair() for _ in range(one_time_keys)]
        
        digest = hash_digest(Streebog, pickle.dumps(self._signed_prekey.public))
        self._signature = gost3410.sign(Curve.CURVE, self._identity_key.private, digest)
        
        if fake_ik:
            #
            # Демонстрация неудачного выполнения, заменю идентификационный ключ
            #
            
            self._identity_key  = SignalClient._generate_key_pair()
        
        self._sessions = {}
        
        trace('[SignalClient]', f'Identity key: {self._identity_key}')
        trace('[SignalClient]', f'Signed prekey: {self._signed_prekey}')
        trace('[SignalClient]', f'Signature: {as_hex(self._signature)}')
        trace('[SignalClient]', f'One time keys: {len(self._ot_prekeys)}')
        
        
    def send_message(self, phone_number: str, message: str, server: SignalServer):
        """
        Отправка сообщения, установление сессии, если нужно
        """
        
        if phone_number not in self._sessions:
            #
            # Сессии еще нет, поэтому установлю соединение
            #
            
            return self._x3dh_init_session(phone_number, message, server)
        
        #
        # Так как протокол X3DH асинхронный, то я прямо сразу и отправлю сообщение
        #
        
        if self._sessions[phone_number].received:
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            self._sessions[phone_number].received = False
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key   = self._sessions[phone_number].dh_ratchet.step()
            
            trace('[SignalClient]', 'Root KDF chain step')
            send_key = self._sessions[phone_number].root_kdf_chain.step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Sending ratchet reset')
            self._sessions[phone_number].send_ratchet.reset(send_key)
            
        #
        # Вырабатывается ключ с помощью симметричного храповика для отправки
        # На ключе шифруется само сообщение
        #
        
        trace('[SignalClient]', 'Sending ratchet step')
        message_key = self._sessions[phone_number].send_ratchet.step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as encryption key')
        encrypted_message = KuznyechikMGM(message_key).encrypt(message.encode())
        
        message_id = self._sessions[phone_number].msg_id
        public_key = self._sessions[phone_number].dh_ratchet.public_key
        
        self._sessions[phone_number].msg_id += 1
        
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     message_id, 
                                                                     encrypted_message))
        
        
    def receive_message(self, phone_number: str, server: SignalServer):
        """
        Прием сообщения, установление сессии, если надо
        """
        
        message = server.peek_message(phone_number, self.phone_number)
        if message is None:
            trace('[SignalClient]', f'No message for {self.phone_number} from {phone_number}')
            return
        
        public_key, message_id, data = message
        
        trace('[SignalClient]', f'Received message with id {message_id} from {phone_number}')
        
        if message_id == SignalClient.X3DH_MSG_ID:
            #
            # Это инициирующее сообщения для установки сессии
            #
            
            return self._x3dh_accept_session(phone_number, data, server)
        
        if message_id == SignalClient.DH_RATCHET_PK_ID:
            #
            # Это нам ключ открытый прислали для инициализации
            #
            
            if not self._sessions[phone_number].dh_ratchet.is_uninitialized():
                raise RuntimeError(f'Cannot initialize DH ratchet for {self.phone_number} twice')
                
            return self._sessions[phone_number].dh_ratchet.initialize(public_key)
        
        #
        # Во-первых, факт приема зафиксирую 
        #
        
        self._sessions[phone_number].received = True
        
        #
        # Не сменился ли открытый ключ собеседника?
        #
        
        if not self._sessions[phone_number].dh_ratchet.is_same_public_key(public_key):
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key = self._sessions[phone_number].dh_ratchet.step(public_key)
            self._sessions[phone_number].dh_ratchet.generate_new_key_pair()
            
            trace('[SignalClient]', 'Root KDF chain step')
            recv_key = self._sessions[phone_number].root_kdf_chain.step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Receiving ratchet reset')
            self._sessions[phone_number].recv_ratchet.reset(recv_key)
            
        #
        # Делаю шаг храповика приема
        #
            
        trace('[SignalClient]', 'Receiving ratchet step')
        message_key = self._sessions[phone_number].recv_ratchet.step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as decryption key')
        decrypted_message = KuznyechikMGM(message_key).decrypt(*data)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
            
        return decrypted_message.decode()
        
        
    @property
    def phone_number(self):
        """
        Получение номер телефона
        """
        
        return self._phone_number
    
    
    @property
    def public_keys(self):
        """
        Получение открытых ключей
        """
        
        return KeyBundle(self._identity_key.public, self._signed_prekey.public, 
                         self._signature, [prekey.public for prekey in self._ot_prekeys])
        
        
    def _x3dh_init_session(self, phone_number: str, message: str, server: SignalServer):
        IKb, SPKb, Sigb, (OPKb, idx) = server.get_public_keys(phone_number)
        
        #
        # Проверяю подпись
        #
        
        digest = hash_digest(Streebog, pickle.dumps(SPKb))
        if not gost3410.verify(Curve.CURVE, IKb, digest, Sigb):
            trace('[SignalClient]', f'Cannot verify signature of client with phone {phone_number}')
            return
        
        #
        # Эфемерный ключ
        #
        
        EKa = SignalClient._generate_key_pair()
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._identity_key, SPKb)
        DH2 = SignalClient._dh(EKa, IKb)
        DH3 = SignalClient._dh(EKa, SPKb)
        DH4 = SignalClient._dh(EKa, OPKb)
        
        #
        # Вырабатываю ключ и создаю сообщение
        # В качестве данных я отправлю шифртекст сообщения,
        # идентификационный ключ, эфемерный ключ и индекс
        # использованного одноразового ключа
        #
        # Протокол разрешает использовать выработанный ключ прямо
        # для зашифрования первого сообщения: An initial ciphertext 
        # encrypted with some AEAD encryption scheme [4] using AD 
        # as associated data and using an encryption key which is 
        # either SK or the output from some cryptographic PRF keyed by SK.
        #
        
        AD = pickle.dumps(self._identity_key.public) + pickle.dumps(IKb)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        #
        # А теперь требуется создать состояние
        #
        
        self._initialize_state(phone_number, root_key)
        
        encrypted_message = KuznyechikMGM(root_key).encrypt(message.encode(), AD)
        message = (encrypted_message, self._identity_key.public, EKa.public, idx)
        
        server.push_message(self.phone_number, phone_number, 
                            Message(None, SignalClient.X3DH_MSG_ID, message))
        
    
    def _x3dh_accept_session(self, phone_number: str, x3dh_parameters: Message, server: SignalServer):
        encrypred_message, IKa, EKa, idx = x3dh_parameters
        
        #
        # Получу нужный одноразовый ключ
        #
        
        OPKb = self._ot_prekeys.pop(idx)
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._signed_prekey, IKa)
        DH2 = SignalClient._dh(self._identity_key, EKa)
        DH3 = SignalClient._dh(self._signed_prekey, EKa)
        DH4 = SignalClient._dh(OPKb, EKa)
        
        #
        # Вырабатываю ключ и читаю сообщение
        #
        
        AD = pickle.dumps(IKa) + pickle.dumps(self._identity_key.public)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        decrypted_message = KuznyechikMGM(root_key).decrypt(*encrypred_message, AD)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
        
        #
        # Теперь нужно инициализировать состояние и отослать открытый ключ
        #
        
        self._initialize_state(phone_number, root_key)
        
        public_key = self._sessions[phone_number].dh_ratchet.public_key
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     SignalClient.DH_RATCHET_PK_ID, 
                                                                     None))
        
        return decrypted_message.decode()
        
        
    def _initialize_state(self, phone_number: str, root_key):
        self._sessions[phone_number] = SignalClient.SessionState(True, 
                                                                 DiffieHellmanRatchet(), 
                                                                 SymmetricRatchet(root_key), 
                                                                 KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                                                 KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                                                 0)
        
        
    def _try_mimic(self, phone_number, session_phone_number, session):
        self._phone_number = phone_number
        self._sessions[session_phone_number] = session
        self._sessions[session_phone_number].dh_ratchet.generate_new_key_pair()
        
        trace('[SignalClient]', f'User now tries to be a user with phone {phone_number}')
        
        
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return KeyPair(private_key, public_key)
    
    
    @staticmethod
    def _dh(key_pair, public_key):
        return Curve.multiply_by_scalar(key_pair.private, public_key)
    
    
    @staticmethod
    def _produce_root_key(DH1, DH2, DH3, DH4):
        kdf_data = pickle.dumps(DH1) + pickle.dumps(DH2) + pickle.dumps(DH3) + pickle.dumps(DH4)
        return HKDF(kdf_data, KuznyechikMGM.KEY_SIZE, b'\x00' * 16, Streebog, 1)

----
## Демонстрация удачного выполнения протоколов

In [18]:
# Создаю сервер
srv = SignalServer()

In [19]:
# Пользователь Алиса
alice = SignalClient('+7 921 059-87-72')

[TRACE] [SignalClient] Identity key: KeyPair(private=12442124307749309071029862161873168184498515238008143490796295161666719857316330431866396065862152165792723434406631270709727659328633545006723406243977891, public=(5565860052959195528411498999526449300214300079280127506989263797804322253523492503146052371121831654246056292351741969042512252341592128694608037157021032, 870277569380833946177765812408107024256062607492550211218701985760450390135753087525592147115523427092421492543884986841501804561127611675757429091876155))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=11292229127230130269545525999935181654522332578128516886494610243151213995239227752369278076149947130307355101945512888916387955980059522717106094063101269, public=(10391610020728069510088615374034402382983308456722903346947393517777520807069132737012178469654273856089500733772760133057383406015421907995928803288938762, 1212347674199588311683246657498030371369411550915263098358704115414320013573021479897829470701

In [20]:
# Пользователь Боб
bob = SignalClient('+7 991 174-51-34')

[TRACE] [SignalClient] Identity key: KeyPair(private=4856257525129982153604500927389574709544329993920982716396188279476381306905451283143331405065202563905012772337725242928044818912198255015662977129978622, public=(7108367292483862894274207625529550008194931729673397080442636384828835687755598189855404620980457818150401813781443091083064684248980477733275358342749922, 1387188628660597990866764595636074140064408442770002060708855062969239154054703897455437769377918512302720922881951661194629907585471379142729688546295003))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=5521738976296579535185272296986246797917360835342070478187346005858113126379759458082279749941558596265488007032296404968759331919021154701039914671387236, public=(9108264195848112585904173823909668519677900490989051627962028006154362343982971373948660851238804674669058652176873175602377245093735564254629256583974431, 105256154437613976853402982819170078774623217726673897837782977314028348099621384063911360597260

In [21]:
# Регистрирую всех 
srv.register(alice.phone_number, alice.public_keys)
srv.register(bob.phone_number, bob.public_keys)

[TRACE] [SignalServer] Registered client with phone number +7 921 059-87-72
[TRACE] [SignalServer] Registered client with phone number +7 991 174-51-34


In [22]:
# Алиса пытается пообщаться с Бобом
alice.send_message(bob.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 1
[TRACE] [SignalClient] Use 80049587000000000000008a406825e0b96a7a21df64c7cf28936e920b07f47a847d43b584e689e030725709b2b53cdae898467143b5a47c51680a46c9aeb75183a6fe38aabb98ad8c735b456a8a403b3d66d1aefeb9e94983ff19811c9e652e63f42422acd58a955a4378e8592a726e6a44385e05aff29ac07740032b673492c79645cf166b5c04135c9001d49d1086942e80049588000000000000008a41e21e65a232da79a5e67c77efdb3c79438e908d45dfb3950ddcb588e9ee001c166ed1c91a68ecf586c37f4a34e536234ca2dd1c78bffb023eafae6280bef9b887008a40db3c20cbac659860d2717db581063c4a72e88768203fd63460874d908ab6fb82d2a568a55eb13a150c71d869d2cef8d16ac39e379f185c1794540186d06f7c1a86942e as associated data
[TRACE] [SignalClient] Use fd01fcd721e50e83b13b9b1914b0a829f7b4e19f9a5d1c707026d939cc7bd6cb as root key
[TRACE] [SignalServer] Saved message with id X3DH_MSG from +7 921 059-87-72 to +7 991 174-51-34


In [23]:
# Боб полчает сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049587000000000000008a406825e0b96a7a21df64c7cf28936e920b07f47a847d43b584e689e030725709b2b53cdae898467143b5a47c51680a46c9aeb75183a6fe38aabb98ad8c735b456a8a403b3d66d1aefeb9e94983ff19811c9e652e63f42422acd58a955a4378e8592a726e6a44385e05aff29ac07740032b673492c79645cf166b5c04135c9001d49d1086942e80049588000000000000008a41e21e65a232da79a5e67c77efdb3c79438e908d45dfb3950ddcb588e9ee001c166ed1c91a68ecf586c37f4a34e536234ca2dd1c78bffb023eafae6280bef9b887008a40db3c20cbac659860d2717db581063c4a72e88768203fd63460874d908ab6fb82d2a568a55eb13a150c71d869d2cef8d16ac39e379f185c1794540186d06f7c1a86942e as associated data
[TRACE] [SignalClient] Use fd01fcd721e50e83b13b9b1914b0a829f7b4e19f9a5d1c707026d939cc7bd6cb as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 991 174-51-34 to +7 921 059-87-72


'Hi!'

In [24]:
# Это небольшой костыль, тут я получаю инициализирующее 
# сообщение с открытым ключом Боба
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id DH_RATCHET_PK from +7 991 174-51-34


In [25]:
# Алиса отсылает сообщение
alice.send_message(bob.phone_number, "What's up?", srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use c0fbd71ba9580099ad82070f60905b98a265c3a7b654b02d563eedeec13a8faa as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 921 059-87-72 to +7 991 174-51-34


In [26]:
# Боб принимает и расшифровывает
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use c0fbd71ba9580099ad82070f60905b98a265c3a7b654b02d563eedeec13a8faa as decryption key


"What's up?"

In [27]:
# Можно послать несколько сообщений подряд
bob.send_message(alice.phone_number, 'Some dummy phrase just to continue the dialog', srv)
bob.send_message(alice.phone_number, 'And another one', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use e2862b80121b534a940415a04701ab030e2c40ae5a6909f2a3af5ded8cc722ed as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 991 174-51-34 to +7 921 059-87-72
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 09e1464655e12892b4deacc68cdb3d46ab9ca5944684445f28192dc16a0ca061 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 991 174-51-34 to +7 921 059-87-72


In [28]:
# И получить оба. Одно...
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use e2862b80121b534a940415a04701ab030e2c40ae5a6909f2a3af5ded8cc722ed as decryption key


'Some dummy phrase just to continue the dialog'

In [29]:
# ... за другим
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 991 174-51-34
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 09e1464655e12892b4deacc68cdb3d46ab9ca5944684445f28192dc16a0ca061 as decryption key


'And another one'

In [30]:
# Можно снова сменить направление отправки и 
# посмотреть, что произошел шаг DH-храповика
alice.send_message(bob.phone_number, 'Yet another DH ratchet step demo', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 0cdded156016e61ad1a8863fac3437bdbe4c4f33fc928fd4d48aafccf2849898 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 921 059-87-72 to +7 991 174-51-34


In [31]:
# А на другой строне надо получить сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 0cdded156016e61ad1a8863fac3437bdbe4c4f33fc928fd4d48aafccf2849898 as decryption key


'Yet another DH ratchet step demo'

In [32]:
# Можно третьего пользователя создать
carol = SignalClient('+7 970 505-44-25')
srv.register(carol.phone_number, carol.public_keys)

[TRACE] [SignalClient] Identity key: KeyPair(private=7738834059388650776104464738803049002401276913744716829640089312760312347311107202249135191390216708892650103928797530126177443030687023259314129838171087, public=(2679034846012206711192272854216388977763728692022387112666767967874128581631321320660070101886771870940867900347530157806891134857154689785504944311547160, 8423883605996883012177975558520986810095904934850089952653349897088092953707254381097881109545382530883606356471029241908042112347648957576877359808780718))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=13049916981104646880607601673276721953495406482850983508125536820648133693481585105187851085589172889812783684621899743466734428097787576014629352047888640, public=(11290128613746124828398786439410540815877541915625671462217178999476580331622833094477448612342380156495574692617818827220070058441631911968540534265772745, 1137632660618847217398429025892131761507740703366733015430355670874886351388724337243137164541

In [33]:
alice.send_message(bob.phone_number, "I'm making new friends", srv)
alice.send_message(carol.phone_number, 'Hi, Carol!', srv)

[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 071e3a9c10474d00718dc344f8e5b27c24e071870054a61c47f3176a81a8536a as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 921 059-87-72 to +7 991 174-51-34
[TRACE] [SignalServer] Using OPK at 1
[TRACE] [SignalClient] Use 80049587000000000000008a406825e0b96a7a21df64c7cf28936e920b07f47a847d43b584e689e030725709b2b53cdae898467143b5a47c51680a46c9aeb75183a6fe38aabb98ad8c735b456a8a403b3d66d1aefeb9e94983ff19811c9e652e63f42422acd58a955a4378e8592a726e6a44385e05aff29ac07740032b673492c79645cf166b5c04135c9001d49d1086942e80049588000000000000008a4018b564601df4f3478d21a682cbbb082ae220fe48318aeb979cd410104eb14519bc103f601e22d70ea9dfe15d3bd3746e92411c255100f7ec93ded7417cd926338a41ae19327f90ac5ca36e2a67270f4adc4799f499c61107427022b2f8fbb314cff1991c63f22b069644f2196244bbaeebceb7caea57399c13ff8d391423da15d7a00086942e as associated data
[TRACE] [SignalClient] Use 9af17713a2e6726860faba0a4bd9d4a1e44cc6ec750ff482d82ef896e58

In [34]:
carol.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049587000000000000008a406825e0b96a7a21df64c7cf28936e920b07f47a847d43b584e689e030725709b2b53cdae898467143b5a47c51680a46c9aeb75183a6fe38aabb98ad8c735b456a8a403b3d66d1aefeb9e94983ff19811c9e652e63f42422acd58a955a4378e8592a726e6a44385e05aff29ac07740032b673492c79645cf166b5c04135c9001d49d1086942e80049588000000000000008a4018b564601df4f3478d21a682cbbb082ae220fe48318aeb979cd410104eb14519bc103f601e22d70ea9dfe15d3bd3746e92411c255100f7ec93ded7417cd926338a41ae19327f90ac5ca36e2a67270f4adc4799f499c61107427022b2f8fbb314cff1991c63f22b069644f2196244bbaeebceb7caea57399c13ff8d391423da15d7a00086942e as associated data
[TRACE] [SignalClient] Use 9af17713a2e6726860faba0a4bd9d4a1e44cc6ec750ff482d82ef896e585db7c as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 970 505-44-25 to +7 921 059-87-72


'Hi, Carol!'

In [35]:
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 921 059-87-72
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 071e3a9c10474d00718dc344f8e5b27c24e071870054a61c47f3176a81a8536a as decryption key


"I'm making new friends"

----
## Демонстрация неуспешного выполнения протоколов

In [36]:
# Пользователь, который как-то украл подпись и подписанный ключ
# Но его идентификационный ключ не позволяет проверить подпись
mallory = SignalClient('+1 202-918-2132', fake_ik=True)

[TRACE] [SignalClient] Identity key: KeyPair(private=4384742645893493514160695085147557174497821900204255180843018134137592105750154378340910765329041667867903915319809292468715906565656080936632277667031166, public=(8889684924892548420026438839826749769540943140073500690026926172755847528900873598809859855605187264782955862958927023509124572116166790464345235221544192, 6356464485786319456169464725766735744309572617435990596676997950501522133629997000573321730427309138362670671209999626212534112621724184634475676048539045))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=6951332178063369763514985722604178685180935232858777003356257794951697736712552278474908191779522605309190391191631520573649176623098610268142467955868820, public=(9944739640659800912360562019122575624801842662543779351960474061034200182050160289422782727895887769634043777723143132821200596467196331674042526058101989, 113059516097223336179902400141640739250606607382502514575498409285607794545507014224200141876110

In [37]:
# Надо зарегистрироваться
srv.register(mallory.phone_number, mallory.public_keys)

[TRACE] [SignalServer] Registered client with phone number +1 202-918-2132


In [38]:
# Алиса получает ключи с сервера и обнаруживает несоответствие
alice.send_message(mallory.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 8
[TRACE] [SignalClient] Cannot verify signature of client with phone +1 202-918-2132


In [39]:
# Еще один пользователь-враг -- Ева
eve = SignalClient('+1 539-567-2473')

[TRACE] [SignalClient] Identity key: KeyPair(private=4983900448805121562366832856099472522805052485061680064563798153959354352279443014766130999465334968034250282001346098422181171341046842144648450873475703, public=(2034220974807066331406490423963012517805540519733673266797528971851869943616874957737432360355151249000506679265490340161918788422569213228065347179167862, 8250802430341286356427742254767897726707668804872046910966047951063340083922618306902726628284923004344691753946148132023302823238612333344827556826442552))
[TRACE] [SignalClient] Signed prekey: KeyPair(private=8537974303592014130390144267804800519175209600302751016198657981712067193753481164986992365133202278078894395335885155253038554854833128866987562011497441, public=(4090056598343417870103164192380664203421090193514580585326452004084956729715496361017028144869449645298860509955871682183165141324490739416783680930631071, 571880666850268454129514671107163996440286043499146162752995437372760942438125629458946295413600

In [40]:
# Она пытается прикинуться Алисой в разговоре с Бобом
eve._try_mimic(alice.phone_number, bob.phone_number, 
               alice._sessions[bob.phone_number])

[TRACE] [SignalClient] User now tries to be a user with phone +7 921 059-87-72


In [41]:
# Боб отправляет Алисе соообщение
bob.send_message(alice.phone_number, 'Hi!', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 0cc7a5a7a52b776cca9ca914da8f5cd2e9806eb1b5ce933618642cc1c47da181 as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 991 174-51-34 to +7 921 059-87-72


In [42]:
# А вот получить его пытается Ева, но расшифровать не получится
eve.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 0d51c079065065b7e29537204fc7f0f8b4ae77eca9f84fca4eda1da5411126f8 as decryption key
[TRACE] [SignalClient] Cannot decrypt message
